# Basic Network Optimization in PyPSA

This tutorial covers how to perform basic network optimization in PyPSA. We'll learn about different optimization problems, constraints, and how to analyze the results.

## Types of Optimization

PyPSA supports several types of optimization problems:

1. **Power Flow (PF)**: Determines power flows in a network given fixed generation and demand
2. **Linear Optimal Power Flow (LOPF)**: Optimizes generation dispatch to minimize costs
3. **Investment Optimization**: Optimizes both investment and operation

Let's create a network and explore these different optimization types:

In [ ]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create network with time series
network = pypsa.Network()
network.set_snapshots(pd.date_range("2024-01-01", periods=24, freq="H"))

# Add buses
for i in range(3):
    network.add("Bus", f"bus {i}", v_nom=20.0)

# Add carriers
network.add("Carrier", "gas", co2_emissions=0.2)
network.add("Carrier", "solar")
network.add("Carrier", "wind")

## Power Flow Optimization

First, let's set up a network for power flow analysis:

In [ ]:
# Add generators with fixed dispatch
network.add(
    "Generator",
    "gas_plant",
    bus="bus 0",
    p_nom=100,
    p_set=80,  # Fixed dispatch
    carrier="gas"
)

# Add renewable generators with variable output
solar_profile = pd.Series(
    [0.0, 0.0, 0.0, 0.0, 0.1, 0.3, 0.5, 0.7, 0.8, 0.9, 0.9, 0.8,
     0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    index=network.snapshots
)

network.add(
    "Generator",
    "solar",
    bus="bus 1",
    p_nom=50,
    p_set=solar_profile * 50,  # Time-varying dispatch
    carrier="solar"
)

# Add load
network.add(
    "Load",
    "load",
    bus="bus 2",
    p_set=50
)

# Add transmission lines
for i in range(3):
    network.add(
        "Line",
        f"line_{i}",
        bus0=f"bus {i}",
        bus1=f"bus {(i + 1) % 3}",
        s_nom=100,
        x=0.1,
        r=0.01
    )

# Solve power flow
network.pf()

# Plot power flows
plt.figure(figsize=(10, 6))
network.lines_t.p0.plot()
plt.title('Power Flows from Power Flow Analysis')
plt.xlabel('Time')
plt.ylabel('Power Flow (MW)')
plt.legend(title='Line')
plt.grid(True)
plt.show()

## Linear Optimal Power Flow (LOPF)

Now let's set up the network for LOPF optimization:

In [ ]:
# Modify generators for LOPF
network.generators.loc["gas_plant", "p_set"] = None  # Remove fixed dispatch
network.generators.loc["gas_plant", "marginal_cost"] = 50  # Add marginal cost
network.generators.loc["solar", "p_set"] = None  # Remove fixed dispatch
network.generators.loc["solar", "marginal_cost"] = 0  # Zero marginal cost for renewables

# Add time-varying load
load_profile = pd.Series(
    [0.6, 0.5, 0.4, 0.4, 0.5, 0.7, 0.8, 0.9, 0.9, 0.8, 0.7, 0.7,
     0.7, 0.7, 0.7, 0.7, 0.8, 0.9, 0.9, 0.8, 0.7, 0.6, 0.5, 0.5],
    index=network.snapshots
)

# Set time-varying load correctly
for i, snapshot in enumerate(network.snapshots):
    network.loads_t.p_set.loc[snapshot, "load"] = 50 * load_profile[snapshot]

# Solve LOPF
network.lopf()

# Plot generator dispatch
plt.figure(figsize=(10, 6))
network.generators_t.p.plot()
plt.title('Generator Dispatch from LOPF')
plt.xlabel('Time')
plt.ylabel('Power (MW)')
plt.legend(title='Generator')
plt.grid(True)
plt.show()

## Investment Optimization

Let's modify the network for investment optimization:

In [ ]:
# Make generators extendable
network.generators.loc["gas_plant", "p_nom_extendable"] = True
network.generators.loc["gas_plant", "capital_cost"] = 1000
network.generators.loc["solar", "p_nom_extendable"] = True
network.generators.loc["solar", "capital_cost"] = 800

# Make lines extendable
network.lines["s_nom_extendable"] = True
network.lines["capital_cost"] = 1000

# Add global constraint for CO2 emissions
network.add(
    "GlobalConstraint",
    "co2_limit",
    sense="<=",
    constant=1000  # Maximum CO2 emissions in tCO2
)

# Solve investment optimization
network.lopf()

# Print optimal capacities
print("\nOptimal Generator Capacities:")
print(network.generators.p_nom_opt)

print("\nOptimal Line Capacities:")
print(network.lines.s_nom_opt)

## Analyzing Results

Let's analyze some key results from the optimization:

In [ ]:
# Calculate system costs
print("\nSystem Costs:")
print(network.objective)

# Calculate generator statistics
print("\nGenerator Statistics:")
print(network.statistics.supply())

# Calculate line statistics
print("\nLine Statistics:")
print(network.statistics.transmission())

# Plot generator dispatch
plt.figure(figsize=(10, 6))
network.generators_t.p.plot()
plt.title('Optimal Generator Dispatch')
plt.xlabel('Time')
plt.ylabel('Power (MW)')
plt.legend(title='Generator')
plt.grid(True)
plt.show()

## Key Optimization Concepts

1. **Power Flow (PF)**
   - Fixed generation and demand
   - Calculates power flows and voltages
   - Uses Newton-Raphson method

2. **Linear Optimal Power Flow (LOPF)**
   - Optimizes generation dispatch
   - Minimizes operational costs
   - Uses linear programming

3. **Investment Optimization**
   - Optimizes both investment and operation
   - Minimizes total system costs
   - Can include constraints (e.g., CO2 limits)

## Next Steps

In the next tutorial, we'll explore more advanced optimization features, including investment planning and multi-period optimization.